In [1]:
from sympy import *
import numpy as np

## (Афанасьев В.Н.) Математическая теория конструирования систем управления $-$ линейная система

In [76]:
n = 3
n_time = 1

t = var("t", real=True)
w = var("w", real=True, constant=True)

def get_vars(name: str, n: int):
    s = ""
    for i in range(n):
        s += f"{name}_{i} "
    return var(s, real=True)

def get_func(name: str, n: int):
    return [Function(f"{name}_{i}")(t) for i in range(n)]

t_ = get_vars("t", n_time)
x = [get_func(f"x^{j}", n) for j in range(n_time)]
y = [get_func(f"y^{j}", n) for j in range(n_time)]
z = [get_func(f"z^{j}", n) for j in range(n_time)]
vx = [get_func(f"v^x{j}", n) for j in range(n_time)]
vy = [get_func(f"v^y{j}", n) for j in range(n_time)]
vz = [get_func(f"v^z{j}", n) for j in range(n_time)]
r = [[sqrt(x[j][i]**2 + y[j][i]**2 + z[j][i]**2) for i in range(n)] for j in range(n_time)]
X_ = []
for i in range(n):
    X_ += [x[0][i], y[0][i], z[0][i], vx[0][i], vy[0][i], vz[0][i]]
# X_ = Matrix(X_)
X_

[x^0_0(t),
 y^0_0(t),
 z^0_0(t),
 v^x0_0(t),
 v^y0_0(t),
 v^z0_0(t),
 x^0_1(t),
 y^0_1(t),
 z^0_1(t),
 v^x0_1(t),
 v^y0_1(t),
 v^z0_1(t),
 x^0_2(t),
 y^0_2(t),
 z^0_2(t),
 v^x0_2(t),
 v^y0_2(t),
 v^z0_2(t)]

In [77]:
Y_ = []
for j in range(n_time):
    R = Matrix([[r[j][i] if i==jj else sqrt((x[j][i]-x[j][jj])**2 + (y[j][i]-y[j][jj])**2 + (z[j][i]-z[j][jj])**2) for i in range(n)] for jj in range(n)])
    Y_ += r[j].copy()
    for i in range(n):
        for j in range(i):
            Y_ += [R[i, j]]
# Y_ = Matrix(Y_)
# ro = get_vars("rho", len(R_))
print(f"Неизвестные: {len(X_)}, Известные: {len(Y_) / n_time}, k={len(X_) / (len(Y_) // n_time)}")
l = int(len(Y_) // n_time)
k = int(len(X_) // (len(Y_) // n_time))
# Y_

Неизвестные: 18, Известные: 6.0, k=3.0


In [92]:
def my_diff(expr, power: int = 1, vari: any = t):
    if power == 0:
        return expr
    if power == 1:
        anw = expr
    else:
        anw = my_diff(expr, power - 1)
    subses = []
    for i in range(n):
        for j in range(n_time):
            subses += [(Derivative(x[j][i], t), vx[j][i])]
            subses += [(Derivative(y[j][i], t), vy[j][i])]
            subses += [(Derivative(z[j][i], t), vz[j][i])]
            subses += [(Derivative(vx[j][i], t), -2*w*vz[j][i] - 1e-6)]
            subses += [(Derivative(vy[j][i], t), -w**2*y[j][i])]
            subses += [(Derivative(vz[j][i], t), 2*w*vx[j][i] + 3*w**2*z[j][i])]
    return anw.diff(vari).subs(subses).simplify()

In [93]:
# H_ = Matrix([[my_diff(Y_[ll], kk) for ll in range(l)] for kk in range(k)])
H_ = Matrix([[Y_[0] for ll in range(l)] for kk in range(k)])
H_one_line = []
for kk in range(k):
    for ll in range(l):
        tmp = Y_[ll] if kk==0 else my_diff(H_[kk - 1, ll])
        H_[kk, ll] = tmp
        H_one_line += [tmp]
# H_

In [116]:
shape(H_)

(3, 6)

## $\to$checkpoint

In [114]:
A_single = Matrix([[0, 0, 0, 1, 0, 0], 
                   [0, 0, 0, 0, 1, 0],
                   [0, 0, 0, 0, 0, 1],
                   [0, 0, 0, 0, 0, -2*w],
                   [0, -w**2, 0, 0, 0, 0],
                   [0, 0, 3*w**2, 2*w, 0, 0]])
A_ = Matrix([[A_single if i==j else zeros(6, 6) for j in range(n)] for i in range(n)])

In [115]:
anw = Matrix.hstack(H_, 
                    A_.T @ H_, 
                    A_.T @ A_.T @ H_, 
                    A_.T @ A_.T @ A_.T @ H_, 
                    A_.T @ A_.T @ A_.T @ A_.T @ H_, 
                    A_.T @ A_.T @ A_.T @ A_.T @ A_.T @ H_)
shape(anw)

ShapeError: Matrix size mismatch: (18, 18) * (3, 6).

In [108]:
Radius_orbit = 6800e3
mu = 5.972e24 * 6.67408e-11
w_0 = np.sqrt(mu / Radius_orbit ** 3)

x0rand = [np.random.uniform(-100, 100) for _ in range(n)]
y0rand = [np.random.uniform(-100, 100) for _ in range(n)]
z0rand = [np.random.uniform(-100, 100) for _ in range(n)]
vx0rand = [np.random.uniform(-1, 1) for _ in range(n)]
vy0rand = [np.random.uniform(-1, 1) for _ in range(n)]
vz0rand = [np.random.uniform(-1, 1) for _ in range(n)]

In [109]:
subses = [(w, w_0)]
for i in range(n):
    subses += [(x[0][i], x0rand[i]), (y[0][i], y0rand[i]), (z[0][i], z0rand[i]), (vx[0][i], vx0rand[i]), (vy[0][i], vy0rand[i]), (vz[0][i], vz0rand[i])]
anw_numb = anw.subs(subses)
det(anw_numb)

In [110]:
anw_get = anw_numb[:,0:6]
anw_get

Matrix([
[111.920485834732, 0.371205798574056, 0.00408786854282351,                    0,                    0,                    0],
[115.690689327415, 0.945571542542173, 0.00702484508913071, -7.14198782418648e-5, -6.89198573440637e-7, -1.62727028742046e-8],
[69.3527105132341, -0.33995394940928, 0.00197638505615263, 0.000640287688750991,  2.17169867883479e-6,  1.80664255251497e-8],
[220.143810267416,  1.27720496915373,   0.010815389498481,     112.299618896271,     0.37249172485893,   0.0040985662016115],
[56.3423083929516, 0.543700710847932,  0.0128373453763167,     115.690689327415,    0.945571542542173,  0.00702484508913071],
[168.371828254168, 0.571075913837815, 0.00475079741361419,     68.8569993385628,    -0.34282990933196,   0.0019520313874017]])

In [111]:
det(anw_get)

0.0207281102680228

Линейная система с 1 челиком ненаблюдаема

## 038 (Shauying R.K.) Observability of Nonlinear Systems

In [63]:
J_ = Matrix([[Y_[0] for _ in range(k)] for _ in range(k)])
for k1 in range(k):
    for k2 in range(k):
        J_[k1, k2] = my_diff(H_[k1, 0], 1, X_[k2])
# J_

In [64]:
shape(J_)

(6, 6)

In [65]:
J_numb = J_.subs([(x[0][0], x0rand), (y[0][0], y0rand), (z[0][0], z0rand), (vx[0][0], vx0rand), (vy[0][0], vy0rand), (vz[0][0], vz0rand), (w, w_0)])
J_numb

Matrix([
[    0.172667766051081,   -0.861093717642721,    0.478229497211504,                     0,                    0,                     0],
[  0.00604087068534745,  0.00167233271913739, 0.000830085877867968,     0.172667766051081,   -0.861093717642721,     0.478229497211504],
[-0.000147039734430793,   6.9941855645064e-6, -1.80590620483103e-5,    0.0131585997458507,  0.00334466543827478,   0.00127136527040675],
[  3.97157884067362e-6, -1.66422791304654e-6,  1.20257969329942e-6, -0.000436387238237535,  1.87995006483193e-5, -0.000101047039642682],
[ -9.92005862442659e-8,  1.23746724537604e-7, -6.60485526501681e-8,   1.54938724723434e-5, -6.63995277313596e-6,   6.76122672862733e-6],
[-5.76690715048524e-11,  -7.8823349545748e-9,  3.42895106752713e-9,  -4.61658858556289e-7,  6.18888802145556e-7,  -4.16020432808427e-7]])

In [67]:
t = 0
d0 = J_numb.minor(t, t)
print(f"Δ={d0}")

Δ=-3.14683294414133E-25


In [68]:
t = 1
d1 = J_numb.minor(t, t)
print(f"Δ={d1}")

Δ=1.11984235961813E-23


In [69]:
t = 2
d2 = J_numb.minor(t, t)
print(f"Δ={d2}")

Δ=-4.25201368923751E-23


In [70]:
t = 3
d3 = J_numb.minor(t, t)
print(f"Δ={d3}")

Δ=3.41107634358119E-22


In [71]:
t = 4
d4 = J_numb.minor(t, t)
print(f"Δ={d4}")

Δ=6.06319617375816E-20


In [72]:
t = 5
d5 = J_numb.minor(t, t)
print(f"Δ={d5}")

Δ=-6.24132599352045E-20


### Можно ли просто решить систему уравнений?

In [ ]:
n = 2

def get_vars(name: str, n: int):
    s = ""
    for i in range(n):
        s += f"{name}_{i} "
    return var(s, real=True)

x = get_vars("x", n)
y = get_vars("y", n)
z = get_vars("z", n)
r = [sqrt(x[i]**2 + y[i]**2 + z[i]**2) for i in range(n)]
X_ = []
for i in range(n):
    X_ += [x[i], y[i], z[i]]
# X_ = Matrix(X_)
X_

In [ ]:
R = Matrix([[r[i] if i==j else sqrt((x[i]-x[j])**2 + (y[i]-y[j])**2 + (z[i]-z[j])**2) for i in range(n)] for j in range(n)])
R_ = r.copy()
for i in range(n):
    for j in range(i):
        R_ += [R[i, j]]
# R_ = Matrix(R_)
ro = get_vars("rho", len(R_))
R_

In [ ]:
ph0, th0, ph1, th1 = var("varphi_0 theta_0 varphi_1 theta_1", real=True)
equations_angles = [Eq(sin(ph0), y[0] / sqrt(x[0]**2 + y[0]**2)),
    Eq(sin(ph1), y[1] / sqrt(x[1]**2 + y[1]**2)),
    Eq(sin(th0), z[0] / r[0]),
    Eq(sin(th1), z[1] / r[1])]
equations_angles

In [ ]:
print(f"Неизвестные: {len(X_)}, Известные: {len(R_) + 4}")

In [ ]:
equations_distanses = [Eq(ro[i], R_[i]) for i in range(len(R_))]
anw = solve(equations_distanses + equations_angles, X_)
anw

### Решение обратной задачи $R=HX$ -> $\hat{X}=H^{+}R$

In [ ]:
n = 3

def get_vars(name: str, n: int):
    s = ""
    for i in range(n):
        s += f"{name}_{i} "
    return var(s, real=True)

x = get_vars("x", n)
y = get_vars("y", n)
z = get_vars("z", n)
r = [sqrt(x[i]**2 + y[i]**2 + z[i]**2) for i in range(n)]

In [ ]:
R = Matrix([[r[i] if i==j else sqrt((x[i]-x[j])**2 + (y[i]-y[j])**2 + (z[i]-z[j])**2) for i in range(n)] for j in range(n)])
R

In [ ]:
R_ = r.copy()
for i in range(n):
    for j in range(i):
        R_ += [R[i, j]]
R_ = Matrix(R_)
R_

In [ ]:
X_ = []
for i in range(n):
    X_ += [x[i], y[i], z[i]]
X_ = Matrix(X_)
X_

In [ ]:
H_ = Matrix([[R_[i].diff(X_[j]) for i in range(len(R_))] for j in range(3 * n)])
H_

In [ ]:
H_.T * H_

In [ ]:
tmp_0 = H_.T * H_
tmp_0.simplify()
tmp_0

In [ ]:
tmp_1 = tmp_0.inv()
tmp_1.simplify()
tmp_1

In [ ]:
H_pi = (H_.T * H_).inv() * H_.T
H_pi

## Бляха муха оно не инверсируется

### Поиск коэффициентов $C_1...C_6$ по измерениям

$\boldsymbol{r} =
\begin{bmatrix}
    { C_4 -3C_1 \omega_{\text{0}} t_{p} + 2C_2 \cos{\omega_0 t_{p}} -2C_3 \sin{\omega_0 t_{p}}  }\\
    { C_5 \sin{\omega_0 t_{p}} + C_6 \cos{\omega_0 t_{p}} }\\
    { 2C_1 + C_2 \sin{\omega_0 t_{p}} + C_3 \cos{\omega_0 t_{p}}}
\end{bmatrix}$

In [ ]:
n = 3
n_time = 2

def get_vars(name: str, n: int):
    s = ""
    for i in range(n):
        s += f"{name}_{i} "
    return var(s, real=True)

w = var("w", real=True, constant=True)
t_ = get_vars("t", n_time)
C1 = get_vars("C1", n)
C2 = get_vars("C2", n)
C3 = get_vars("C3", n)
C4 = get_vars("C4", n)
C5 = get_vars("C5", n)
C6 = get_vars("C6", n)
C_ = C1 + C2 + C3 + C4 + C5 + C6  # Вектор состояний

R_c = []  # Вектор состояний (переведённый)
for t in range(n_time):
    x_tmp = [C4[i] - 3*C1[i]*w*t_[t] + 2*C2[i]*cos(w*t_[t]) - 2*C3[i]*sin(w*t_[t]) for i in range(n)]
    y_tmp = [C6[i]*cos(w*t_[t]) + C5[i]*sin(w*t_[t]) for i in range(n)]
    z_tmp = [2*C1[i] + C3[i]*cos(w*t_[t]) + C2[i]*sin(w*t_[t]) for i in range(n)]
    R_tmp = [sqrt(x_tmp[i]**2 + y_tmp[i]**2 + z_tmp[i]**2) for i in range(n)]
    R_tt = [[R_tmp[i] if i==j else sqrt((x_tmp[i]-x_tmp[j])**2 + (y_tmp[i]-y_tmp[j])**2 + (z_tmp[i]-z_tmp[j])**2) for i in range(n)] for j in range(n)]
    
    for i in range(n):
        for j in range(i):
            R_tmp += [R_tt[i][j]]
    R_c += R_tmp
len(R_c)

In [ ]:
H_c = Matrix([[R_c[i].diff(C_[j]) for i in range(len(R_c))] for j in range(len(C_))])
H_c

In [ ]:
R_c

## Проба на данных

In [ ]:
n_model = 10

X_model = np.random.uniform(-100, 100, 3 * n_model)
x_model = [X_model[i * 3 + 0] for i in range(n_model)]
y_model = [X_model[i * 3 + 1] for i in range(n_model)]
z_model = [X_model[i * 3 + 2] for i in range(n_model)]
r_model = [sqrt(x_model[i]**2 + y_model[i]**2 + z_model[i]**2) for i in range(n_model)]
R_m = [[r_model[i] if i==j else sqrt((x_model[i]-x_model[j])**2 + (y_model[i]-y_model[j])**2 + (z_model[i]-z_model[j])**2) for i in range(n_model)] for j in range(n_model)]
R_model = r_model.copy()
for i in range(n_model):
    for j in range(i):
        R_model += [R_m[i][j]]
R_model